In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import pandas as pd
import os
import pickle
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
!pip install torch
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 58.5 MB/s eta 0:00:00


In [ ]:
import faiss

In [ ]:
model_name = "Facenet512"
model_name = "ArcFace"
#option_movie_dir = ["like_me", "Calloused_Hands", "Memphis", "Liberty_Kid", "losing_ground"]
#option_movie_dir = "Calloused_Hands"
#option_movie_dir = "losing_ground"
#option_movie_dir = "Memphis"
option_movie_dir = "Liberty_Kid"
#option_movie_dir = "like_me"
#option_character = "Burt"
#option_character = "Kiya"
#option_character = "sara"
#option_character = "Byrd"
#option_character = "Debbie"
#option_character = "willis"
option_character = "Derrick"
Number_Img = 1
#lst_filname_face_query = ["Byrd_1.jpg","Byrd_2.jpg","Byrd_3.jpg","Byrd_4.jpg","Byrd_5.jpg","Byrd_6.jpg"]
#lst_filname_face_query = ["Debbie_1.jpg","Debbie_2.jpg","Debbie_3.jpg","Debbie_4.jpg","Debbie_5.jpg","Debbie_6.jpg"]
#lst_filname_face_query = ["willis_1.jpg","willis_2.jpg","willis_3.jpg","willis_4.jpg","willis_5.jpg","willis_6.jpg","willis_7.jpg"]
#lst_filname_face_query = ["Burt1.jpg","Burt2.jpg","Burt3.jpg","Burt4.jpg","Burt5.jpg"]
lst_filname_face_query = ["Derrick_1.jpg","Derrick_2.jpg","Derrick_3.jpg","Derrick_4.jpg","Derrick_5.jpg","Derrick_6.jpg","Derrick_7.jpg"]
#lst_filname_face_query = ["Kiya1.jpg","Kiya2.jpg","Kiya3.jpg","Kiya4.jpg","Kiya5.jpg"]
#lst_filname_face_query = ["sara_1.jpg","sara_2.jpg","sara_3.jpg","sara_4.jpg","sara_5.jpg"]
root_dataset = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022"

path_to_file_gt= f"{root_dataset}/ground_truth/{option_movie_dir}/{option_character}.xlsx"
path_to_file_index =f"/content/drive/MyDrive/KLTN2022_ThanhHa/source/faiss_index/{model_name}/{option_movie_dir}/{option_movie_dir}.pickle"
path_to_file_index_fnames =f"/content/drive/MyDrive/KLTN2022_ThanhHa/source/faiss_index/{model_name}/{option_movie_dir}/{option_movie_dir}-fnames.pickle"


# Load file index

In [ ]:
index = faiss.read_index(path_to_file_index)

with open(path_to_file_index_fnames, 'rb') as f:
  lst_face_name = pickle.load(f)
list_fname = lst_face_name

# Load ground truth

In [ ]:
gt_path = pd.read_excel(path_to_file_gt)
df_ground_truth_full = gt_path["Full"]
#df_ground_truth_full
lst_groundtruth = list(df_ground_truth_full)

# Đọc feature face query

In [ ]:
lst_name_face_query = []
arr_feature_query = []
for filname_face_query in lst_filname_face_query:
  name_face_query = filname_face_query.split('.')[0]
  path_to_file_feature_query  = f'{root_dataset}/features_query/deepface/{model_name}/{option_movie_dir}/{option_character}/{name_face_query}.pickle'
  path_to_file_face_query  = f'{root_dataset}/faces_query/{option_movie_dir}/{option_character}/{filname_face_query}'
  if os.path.exists(path_to_file_feature_query):
    filename_feature_query = open(path_to_file_feature_query, 'rb')
    feature_query = np.array(pickle.load(filename_feature_query))
    #feature_query = feature_query.reshape(1,-1)
    print(feature_query.shape)
    arr_feature_query.append(feature_query)
    lst_name_face_query.append(name_face_query)

(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)


In [ ]:
arr_feature_query = np.array(arr_feature_query)
arr_feature_query.shape

(7, 512)

In [ ]:
lst_name_face_query

['Derrick_1',
 'Derrick_2',
 'Derrick_3',
 'Derrick_4',
 'Derrick_5',
 'Derrick_6',
 'Derrick_7']

# Thực hiện truy vấn

In [ ]:
#Lấy tất cả kết quả
topk=len(lst_face_name)

In [ ]:
#distance, indices = index.search(feature_query,k=topk)
distance, indices = index.search(arr_feature_query,k=topk)
#print(distance[0])
#print(indices[0])
lst_indices = indices[5]
lst_distance = distance[5]

In [ ]:
indices

array([[14131, 14148,  6760, ..., 10314, 18863, 23402],
       [ 3350,  3347, 23802, ..., 23351, 18863, 23402],
       [34658, 34657, 34661, ..., 10314, 23402, 18863],
       [ 2038,  2040,  2152, ..., 10314, 18863, 23402],
       [ 5761,  5762,  5763, ..., 10314, 18863, 23402],
       [32574, 32377, 32580, ..., 10314, 18863, 23402]])

In [ ]:
distance

array([[  2.0711966,   2.1088955,   2.4152875, ..., 404.52325  ,
        412.63226  , 414.70764  ],
       [  4.860038 ,   4.8927135,   4.9712877, ..., 334.95236  ,
        341.9472   , 344.82007  ],
       [  3.7683487,   4.2647142,   5.1449976, ..., 593.9319   ,
        603.7258   , 603.79175  ],
       [  3.0111227,   4.2040052,   4.5310383, ..., 581.27496  ,
        589.3457   , 590.2809   ],
       [  1.8199186,   1.9030457,   2.2973418, ..., 587.77576  ,
        596.027    , 599.2269   ],
       [  2.914991 ,   2.9902422,   3.005231 , ..., 569.59204  ,
        580.2622   , 580.445    ]], dtype=float32)

In [ ]:
print(lst_indices)

[26506 26509 26508 ... 10874 25799 10873]


In [ ]:
print(lst_distance)

[  1.5790193   1.796623    1.8082521 ... 567.74945   580.9226
 624.4862   ]


In [ ]:
path_to_face = '/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/'
#cv2_imshow(cv2.imread(path_to_file_face_query))
#for fname in list_fname[0:200]:
for ii in lst_indices:
  fname = list_fname[ii]
  arr_tmp = fname.split('-')
  path_fname = os.path.join(path_to_face, arr_tmp[0],f'{arr_tmp[0]}-{arr_tmp[1]}', f'{arr_tmp[0]}-{arr_tmp[1]}-{arr_tmp[2]}', f'{arr_tmp[0]}-{arr_tmp[1]}-{arr_tmp[2]}-{arr_tmp[3]}',fname)
  #print(path_fname)
  cv2_imshow(cv2.imread(path_fname))

 # Chuyển thành ma trận và tạo danh sách kết quả

In [ ]:
MAX_SCENES = 100
MAX_SHOTS = 200

In [ ]:
def Refine_Result(lst_distance, lst_indices, list_fname):
  matrix_distance_min = -np.ones((MAX_SCENES,MAX_SHOTS))
  
  for dist,idx in zip(lst_distance,lst_indices):
    predicted_face = list_fname[idx]
    #print(predicted_face)
    ind = predicted_face.split('-')
    scene_id = int(ind[1])
    shot_id = int(ind[2].split('_')[1])
    #print(f"scene_id={scene_id}, shot_id={shot_id}")    

    distance_min = matrix_distance_min[scene_id, shot_id]
    if distance_min==-1:
      distance_min = dist
    else:
      distance_min = min(dist,distance_min)

    matrix_distance_min[scene_id, shot_id] = distance_min
  return matrix_distance_min

In [ ]:
def Create_List_Results(matrix_distance_min,option_movie_dir):
  lst_results = []
  idx_distance_min = np.argwhere(matrix_distance_min > -1)
  arr_distance = []
  for idx in idx_distance_min:
    arr_distance.append(matrix_distance_min[idx[0], idx[1]])
  arr_distance = np.array(arr_distance)
  idx_sorted = arr_distance.argsort()
  
  for jdx in idx_sorted:
    result = f'{option_movie_dir}-{idx_distance_min[jdx][0]}-shot_{idx_distance_min[jdx][1]}'
    lst_results.append(result)
  return lst_results

**Đối với một face query**

In [ ]:
matrix_distance_min =  Refine_Result(lst_distance,lst_indices,list_fname)
lst_results = Create_List_Results(matrix_distance_min,option_movie_dir)

In [ ]:
lst_results

['Liberty_Kid-47-shot_2',
 'Liberty_Kid-52-shot_14',
 'Liberty_Kid-47-shot_12',
 'Liberty_Kid-33-shot_33',
 'Liberty_Kid-22-shot_7',
 'Liberty_Kid-49-shot_17',
 'Liberty_Kid-21-shot_19',
 'Liberty_Kid-33-shot_29',
 'Liberty_Kid-4-shot_1',
 'Liberty_Kid-22-shot_15',
 'Liberty_Kid-21-shot_7',
 'Liberty_Kid-33-shot_31',
 'Liberty_Kid-9-shot_8',
 'Liberty_Kid-21-shot_10',
 'Liberty_Kid-42-shot_15',
 'Liberty_Kid-52-shot_11',
 'Liberty_Kid-42-shot_12',
 'Liberty_Kid-50-shot_0',
 'Liberty_Kid-25-shot_6',
 'Liberty_Kid-24-shot_18',
 'Liberty_Kid-33-shot_27',
 'Liberty_Kid-10-shot_11',
 'Liberty_Kid-21-shot_3',
 'Liberty_Kid-10-shot_13',
 'Liberty_Kid-38-shot_23',
 'Liberty_Kid-13-shot_20',
 'Liberty_Kid-31-shot_14',
 'Liberty_Kid-22-shot_11',
 'Liberty_Kid-35-shot_2',
 'Liberty_Kid-49-shot_19',
 'Liberty_Kid-24-shot_20',
 'Liberty_Kid-38-shot_11',
 'Liberty_Kid-27-shot_9',
 'Liberty_Kid-4-shot_34',
 'Liberty_Kid-24-shot_16',
 'Liberty_Kid-7-shot_8',
 'Liberty_Kid-25-shot_8',
 'Liberty_Kid-25-

**Đối với nhiều face query**

In [ ]:
lst_matrix_distance_min=[]
lst_lst_results = []
for i in range(indices.shape[0]):
  matrix_distance_min =  Refine_Result(distance[i],indices[i],list_fname)
  
  lst_results = Create_List_Results(matrix_distance_min,option_movie_dir)
  
  lst_matrix_distance_min.append(matrix_distance_min)  
  lst_lst_results.append(lst_results)

In [ ]:
lst_lst_results

[['Calloused_Hands-3-shot_23',
  'Calloused_Hands-2-shot_19',
  'Calloused_Hands-30-shot_15',
  'Calloused_Hands-33-shot_1',
  'Calloused_Hands-22-shot_1',
  'Calloused_Hands-13-shot_2',
  'Calloused_Hands-15-shot_3',
  'Calloused_Hands-41-shot_4',
  'Calloused_Hands-39-shot_0',
  'Calloused_Hands-49-shot_2',
  'Calloused_Hands-57-shot_18',
  'Calloused_Hands-17-shot_8',
  'Calloused_Hands-33-shot_2',
  'Calloused_Hands-9-shot_8',
  'Calloused_Hands-19-shot_16',
  'Calloused_Hands-21-shot_19',
  'Calloused_Hands-38-shot_50',
  'Calloused_Hands-11-shot_21',
  'Calloused_Hands-21-shot_15',
  'Calloused_Hands-51-shot_1',
  'Calloused_Hands-1-shot_6',
  'Calloused_Hands-56-shot_6',
  'Calloused_Hands-3-shot_10',
  'Calloused_Hands-27-shot_4',
  'Calloused_Hands-2-shot_17',
  'Calloused_Hands-34-shot_10',
  'Calloused_Hands-30-shot_17',
  'Calloused_Hands-57-shot_4',
  'Calloused_Hands-27-shot_20',
  'Calloused_Hands-44-shot_9',
  'Calloused_Hands-35-shot_1',
  'Calloused_Hands-27-shot_16',

**Tạo danh sách kết quả cho 2 face query đầu tiên**

In [ ]:
matrix_result = np.zeros((MAX_SCENES,MAX_SHOTS))

for r in range(0,100):
  for c in range(0,200):
    if (lst_matrix_distance_min[0][r][c] == -1) and (lst_matrix_distance_min[1][r][c] == -1):
      matrix_result[r][c] = -1
    if (lst_matrix_distance_min[0][r][c] != -1) and (lst_matrix_distance_min[1][r][c] != -1):
      matrix_result[r][c] = min(lst_matrix_distance_min[0][r][c],lst_matrix_distance_min[1][r][c])
    if (lst_matrix_distance_min[0][r][c] == -1) and (lst_matrix_distance_min[1][r][c] != -1):
      matrix_result[r][c] = lst_matrix_distance_min[1][r][c]
    if (lst_matrix_distance_min[0][r][c] != -1) and (lst_matrix_distance_min[1][r][c] == -1):
      matrix_result[r][c] = lst_matrix_distance_min[0][r][c]

In [ ]:
lst_lst_lst_results = Create_List_Results(matrix_result,option_movie_dir)

In [ ]:
lst_lst_lst_results

['Calloused_Hands-3-shot_23',
 'Calloused_Hands-2-shot_19',
 'Calloused_Hands-30-shot_15',
 'Calloused_Hands-33-shot_1',
 'Calloused_Hands-22-shot_1',
 'Calloused_Hands-13-shot_2',
 'Calloused_Hands-15-shot_3',
 'Calloused_Hands-41-shot_4',
 'Calloused_Hands-39-shot_0',
 'Calloused_Hands-49-shot_2',
 'Calloused_Hands-57-shot_18',
 'Calloused_Hands-17-shot_8',
 'Calloused_Hands-33-shot_2',
 'Calloused_Hands-9-shot_8',
 'Calloused_Hands-19-shot_16',
 'Calloused_Hands-21-shot_19',
 'Calloused_Hands-38-shot_50',
 'Calloused_Hands-11-shot_21',
 'Calloused_Hands-21-shot_15',
 'Calloused_Hands-51-shot_1',
 'Calloused_Hands-1-shot_6',
 'Calloused_Hands-56-shot_6',
 'Calloused_Hands-3-shot_10',
 'Calloused_Hands-27-shot_4',
 'Calloused_Hands-2-shot_17',
 'Calloused_Hands-34-shot_10',
 'Calloused_Hands-30-shot_17',
 'Calloused_Hands-57-shot_4',
 'Calloused_Hands-27-shot_20',
 'Calloused_Hands-44-shot_9',
 'Calloused_Hands-35-shot_1',
 'Calloused_Hands-27-shot_16',
 'Calloused_Hands-13-shot_7',
 

**Tạo danh sách kết quả cho tất cả face query**

In [ ]:
matrix_matrix_result = lst_matrix_distance_min[0]

for j in range(1, len(lst_matrix_distance_min)):
  for r1 in range(MAX_SCENES):
    for c1 in range(MAX_SHOTS):
      if (matrix_matrix_result[r1][c1] == -1) and (lst_matrix_distance_min[j][r1][c1] == -1):
        matrix_matrix_result[r1][c1] = -1
      if (matrix_matrix_result[r1][c1] != -1) and (lst_matrix_distance_min[j][r1][c1] != -1):
        matrix_matrix_result[r1][c1] = min(lst_matrix_distance_min[j][r1][c1],matrix_matrix_result[r1][c1])
      if (matrix_matrix_result[r1][c1] == -1) and (lst_matrix_distance_min[j][r1][c1] != -1):
        matrix_matrix_result[r1][c1] = lst_matrix_distance_min[j][r1][c1]
      if (matrix_matrix_result[r1][c1] != -1) and (lst_matrix_distance_min[j][r1][c1] == -1):
        matrix_matrix_result[r1][c1] = matrix_matrix_result[r1][c1]

In [ ]:
lst_lst_lst_results_1 = Create_List_Results(matrix_matrix_result,option_movie_dir)

In [ ]:
lst_lst_lst_results_1

['Calloused_Hands-19-shot_1',
 'Calloused_Hands-3-shot_23',
 'Calloused_Hands-2-shot_19',
 'Calloused_Hands-30-shot_15',
 'Calloused_Hands-33-shot_1',
 'Calloused_Hands-22-shot_1',
 'Calloused_Hands-13-shot_2',
 'Calloused_Hands-15-shot_3',
 'Calloused_Hands-41-shot_4',
 'Calloused_Hands-39-shot_0',
 'Calloused_Hands-49-shot_2',
 'Calloused_Hands-57-shot_17',
 'Calloused_Hands-57-shot_18',
 'Calloused_Hands-57-shot_13',
 'Calloused_Hands-12-shot_1',
 'Calloused_Hands-17-shot_8',
 'Calloused_Hands-33-shot_2',
 'Calloused_Hands-9-shot_8',
 'Calloused_Hands-19-shot_16',
 'Calloused_Hands-21-shot_19',
 'Calloused_Hands-38-shot_50',
 'Calloused_Hands-11-shot_21',
 'Calloused_Hands-57-shot_15',
 'Calloused_Hands-21-shot_15',
 'Calloused_Hands-51-shot_1',
 'Calloused_Hands-1-shot_6',
 'Calloused_Hands-56-shot_6',
 'Calloused_Hands-3-shot_10',
 'Calloused_Hands-27-shot_4',
 'Calloused_Hands-2-shot_17',
 'Calloused_Hands-34-shot_10',
 'Calloused_Hands-30-shot_17',
 'Calloused_Hands-57-shot_4',


# Đánh giá kết quả

In [ ]:
# Evaluation Metrics For Information Retrieval
def EvaluationMetrics(lst_predict, lst_groundtruth):
    num_gt = len(lst_groundtruth)
    num_correct = 0
    pos_predict = 0
    arr_Precision = []
    arr_Recall = []
    arr_F1 = []
    arr_AP = []
    myAP = 0
    arr_Reciprocal_Rank = []
    
    First_Reciprocal_Rank = -1
    for predict in lst_predict:
        pos_predict = pos_predict +1
        #print(f"predict = {predict}")
        if predict in lst_groundtruth:
            #print(f"predict in lst_groundtruth = {predict}")
            num_correct += 1
            myAP = myAP + num_correct/pos_predict
            Reciprocal_Rank = 1.0/pos_predict

            arr_Reciprocal_Rank.append(Reciprocal_Rank )
            if First_Reciprocal_Rank == -1:
                First_Reciprocal_Rank = Reciprocal_Rank
        else:
            arr_Reciprocal_Rank.append(0)
        
        Precision_k = num_correct/pos_predict
        arr_Precision.append(Precision_k)

        Recall_k = num_correct/num_gt
        arr_Recall.append(Recall_k)
        if Precision_k + Recall_k >0:
          F1_k = 2*Precision_k*Recall_k/(Precision_k+Recall_k)
        else:
          F1_k = 0

        arr_F1.append(F1_k)
        if num_correct>0:
          arr_AP.append(myAP/num_correct)
        else:
          arr_AP.append(0)

    return First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP

**Đối với một face query**

In [ ]:
lst_predict = lst_results

In [ ]:
First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP = EvaluationMetrics(lst_predict, lst_groundtruth)

In [ ]:
print("AP_1=%5.2f" % arr_AP[0], 
      "AP_5=%5.2f" % arr_AP[4],
      "AP_10=%5.2f" % arr_AP[9], 
      "AP_50=%5.2f"% arr_AP[49], 
      "AP_100=%5.2f"% arr_AP[99])

AP_1= 1.00 AP_5= 1.00 AP_10= 1.00 AP_50= 0.99 AP_100= 0.97


**Đối với nhiều face query**

In [ ]:
lst_lst_predict = lst_lst_lst_results_1

In [ ]:
First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP = EvaluationMetrics(lst_lst_predict, lst_groundtruth)

In [ ]:
print("AP_1=%5.2f" % arr_AP[0], 
      "AP_5=%5.2f" % arr_AP[4],
      "AP_10=%5.2f" % arr_AP[9], 
      "AP_50=%5.2f"% arr_AP[49], 
      "AP_100=%5.2f"% arr_AP[99])

AP_1= 1.00 AP_5= 1.00 AP_10= 0.97 AP_50= 0.83 AP_100= 0.74
